# T3. dataloader 的内部结构和基本使用

&emsp; 1 &ensp; fastNLP 中的 dataloader
 
&emsp; &emsp; 1.1 &ensp; dataloader 的职责描述

&emsp; &emsp; 1.2 &ensp; dataloader 的基本使用

&emsp; 2 &ensp; fastNLP 中 dataloader 的延伸

&emsp; &emsp; 2.1 &ensp; collator 的概念与使用

&emsp; &emsp; 2.2 &ensp; sampler 的概念与使用

## 1. fastNLP 中的 dataloader

### 1.1 dataloader 的职责描述

在`fastNLP 0.8`中，在数据加载模块`DataLoader`之前，还存在其他的一些模块，负责例如对文本数据

&emsp; 进行补零对齐，即 **核对器`collator`模块**，进行分词标注，即 **分词器`tokenizer`模块**

&emsp; 本节将对`fastNLP`中的核对器`collator`等展开介绍，分词器`tokenizer`将在下一节中详细介绍

在`fastNLP 0.8`中，**核对器`collator`模块负责文本序列的补零对齐**，通过

### 1.2 dataloader 的基本使用

在`fastNLP 0.8`中，在数据加载模块`DataLoader`之前，还存在其他的一些模块，负责例如对文本数据

&emsp; 进行补零对齐，即 **核对器`collator`模块**，进行分词标注，即 **分词器`tokenizer`模块**

&emsp; 本节将对`fastNLP`中的核对器`collator`等展开介绍，分词器`tokenizer`将在下一节中详细介绍

在`fastNLP 0.8`中，**核对器`collator`模块负责文本序列的补零对齐**，通过

In [ ]:
import pandas as pd
from functools import partial
from fastNLP.transformers.torch import BertTokenizer

from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import DataBundle


class PipeDemo:
    def __init__(self, tokenizer='bert-base-uncased', num_proc=1):
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)
        self.num_proc = num_proc

    def process_from_file(self, path='./data/test4dataset.tsv'):
        datasets = DataSet.from_pandas(pd.read_csv(path))
        train_ds, test_ds = datasets.split(ratio=0.7)
        train_ds, dev_ds = datasets.split(ratio=0.8)
        data_bundle = DataBundle(datasets={'train': train_ds, 'dev': dev_ds, 'test': test_ds})

        encode = partial(self.tokenizer.encode_plus, max_length=100, truncation=True,
                         return_attention_mask=True)
        data_bundle.apply_field_more(encode, field_name='text', num_proc=self.num_proc)

        target_vocab = Vocabulary(padding=None, unknown=None)

        target_vocab.from_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label')
        target_vocab.index_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label',
                                   new_field_name='target')

        data_bundle.set_pad('input_ids', pad_val=self.tokenizer.pad_token_id)
        data_bundle.set_ignore('label', 'text')  
        return data_bundle

&emsp; 

In [ ]:
pipe = PipeDemo(tokenizer='bert-base-uncased', num_proc=4)

data_bundle = pipe.process_from_file('./data/test4dataset.tsv')

&emsp; 

In [ ]:
from fastNLP import prepare_torch_dataloader

dl_bundle = prepare_torch_dataloader(data_bundle, batch_size=arg.batch_size)

&emsp; 

```python
trainer = Trainer(
    model=model,
    train_dataloader=dl_bundle['train'],
    optimizers=optimizer,
	...
	driver="torch",
	device='cuda',
	...
    evaluate_dataloaders={'dev': dl_bundle['dev'], 'test': dl_bundle['test']},     
    metrics={'acc': Accuracy()},
	...
)
```

## 2. fastNLP 中 dataloader 的延伸

### 2.1 collator 的概念与使用

在`fastNLP 0.8`中，在数据加载模块`DataLoader`之前，还存在其他的一些模块，负责例如对文本数据

&emsp; 进行补零对齐，即 **核对器`collator`模块**，进行分词标注，即 **分词器`tokenizer`模块**

&emsp; 本节将对`fastNLP`中的核对器`collator`等展开介绍，分词器`tokenizer`将在下一节中详细介绍

在`fastNLP 0.8`中，**核对器`collator`模块负责文本序列的补零对齐**，通过

In [ ]:
from fastNLP import prepare_torch_dataloader

dl_bundle = prepare_torch_dataloader(data_bundle, train_batch_size=2)

print(type(dl_bundle), type(dl_bundle['train']))

In [ ]:
dataloader = prepare_torch_dataloader(datasets['train'], train_batch_size=2)
print(type(dataloader))
print(dir(dataloader))

In [ ]:
dataloader.collate_fn

### 2.2 sampler 的概念与使用

In [ ]:
dataloader.batch_sampler

&emsp; 